### 2.2 Spectral clustering of the cow data

Learning goal: Idea of spectral embedding (and clustering)

In this task, you should perform 1-dimensional spectral embedding for the cow data, based on Goodall similarity. Since the file cowdist.csv gives the Goodall distance $d_{GO}$, you need to convert it to Goodall similarity by $sim = 1 − d_{GO}$.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

1. **Similarity/Weight Matrix (W)**:
Suppose we have a set of data points, and we compute the pairwise similarities between them to get the similarity matrix. The entry $W_{ij}$ represents the similarity between data points i and j.
For simplicity, let's consider a dataset with 3 points and the following similarity matrix:

$$ W = \begin{bmatrix}
1 & 0.8 & 0.3 \\
0.8 & 1 & 0.5 \\
0.3 & 0.5 & 1 \\
\end{bmatrix} $$

2. **Diagonal Degree Matrix ($\Lambda$)**:
The degree of a node in the similarity graph is the sum of the weights (similarities) of its edges. The degree matrix is a diagonal matrix where the diagonal entry $ \Lambda_{ii} $ is the sum of the $ i^{th} $ row of the similarity matrix.

For our example:
$ \Lambda = \begin{bmatrix}
2.1 & 0 & 0 \\
0 & 2.3 & 0 \\
0 & 0 & 1.8 \\
\end{bmatrix} $, where $ \Lambda_{11} = 2.1 = 1 + 0.8 + 0.3 $, $ \Lambda_{22} = 2.3 = 0.8 + 1 + 0.5 $ and so on

3. **Laplacian Matrix (L)**:
The Laplacian matrix is defined as:
$ L = \Lambda - W $

For our example:
$ L = \begin{bmatrix}
2.1 & 0 & 0 \\
0 & 2.3 & 0 \\
0 & 0 & 1.8 \\
\end{bmatrix} - \begin{bmatrix}
1 & 0.8 & 0.3 \\
0.8 & 1 & 0.5 \\
0.3 & 0.5 & 1 \\
\end{bmatrix} = \begin{bmatrix}
1.1 & -0.8 & -0.3 \\
-0.8 & 1.3 & -0.5 \\
-0.3 & -0.5 & 0.8 \\
\end{bmatrix} $

This Laplacian matrix $L$ captures the structure of the similarity graph. The eigenvalues and eigenvectors of $L$ can be used in spectral clustering to identify clusters in the data.

Note: There are also other variants of the Laplacian matrix, such as the normalized Laplacian, which can be used in spectral clustering. The choice of which Laplacian to use depends on the specific application and dataset.

---


(a) Create a 2-nearest neighbour similarity graph, where the edge weight is the Goodall similarity. Present the corresponding weight matrix $\textbf{W}$.

In [11]:
import pandas as pd
import numpy as np

# Load the cow data
df = pd.read_csv('cowdist.csv')

# Convert Goodall distance to Goodall similarity
df['sim'] = 1 - df['dGO']

# Create a similarity matrix

sim_matrix = np.ones((6, 6))

index_name = {
    "Clover": 0,
    "Sunny": 1,
    "Rose": 2,
    "Daisy": 3,
    "Strawberry": 4,
    "Molly": 5,
}
for index, row in df.iterrows():
    cow1, cow2 = row['pair'][5:-1].split('-')
    i, j = index_name[cow1], index_name[cow2]
    sim_matrix[i][j] = row['sim']
    sim_matrix[j][i] = row['sim']  # since it's symmetric

print("The similarity matrix W is: ")
for i in range(6):
    sim_matrix[i][i] = 0
print(sim_matrix)

# Create a 2-nearest neighbor similarity graph
W = np.zeros((6, 6))

for i in range(6):
    # Get the indices of the two largest similarities for each row
    neighbors = sim_matrix[i].argsort()[-3:-1]  # -3:-1 because the largest similarity is with itself
    for j in neighbors:
        W[i][j] = sim_matrix[i][j]

print()
print("The 2-nearest neighbor similarity graph is: ")
print(W)


The similarity matrix W is: 
[[1.     0.     0.5463 0.2963 0.25   0.    ]
 [0.     1.     0.     0.25   0.     0.5463]
 [0.5463 0.     1.     0.     0.5463 0.    ]
 [0.2963 0.25   0.     1.     0.     0.5463]
 [0.25   0.     0.5463 0.     1.     0.    ]
 [0.     0.5463 0.     0.5463 0.     1.    ]]

The 2-nearest neighbor similarity graph is: 
[[0.     0.     0.5463 0.2963 0.     0.    ]
 [0.     0.     0.     0.25   0.     0.5463]
 [0.5463 0.     0.     0.     0.5463 0.    ]
 [0.2963 0.     0.     0.     0.     0.5463]
 [0.25   0.     0.5463 0.     0.     0.    ]
 [0.     0.5463 0.     0.5463 0.     0.    ]]


(b) Calculate the corresponding (unnormalized) Laplacian matrix $\mathbf{L = \Lambda − W}$, where $\mathbf{\Lambda}$ is the degree matrix.

In [20]:
# Calculate the degree matrix Lambda
Lambda = np.diag(np.sum(W, axis=1))

print("The degree matrix Λ is: ")
print(Lambda)

# Calculate the Laplacian matrix
L = Lambda - W

print("\nThe unnormalized Laplacian matrix L = Λ - W is: ")
print(L)

The degree matrix Λ is: 
[[0.8426 0.     0.     0.     0.     0.    ]
 [0.     0.7963 0.     0.     0.     0.    ]
 [0.     0.     1.0926 0.     0.     0.    ]
 [0.     0.     0.     0.8426 0.     0.    ]
 [0.     0.     0.     0.     0.7963 0.    ]
 [0.     0.     0.     0.     0.     1.0926]]

The unnormalized Laplacian matrix L = Λ - W is: 
[[ 0.8426  0.     -0.5463 -0.2963  0.      0.    ]
 [ 0.      0.7963  0.     -0.25    0.     -0.5463]
 [-0.5463  0.      1.0926  0.     -0.5463  0.    ]
 [-0.2963  0.      0.      0.8426  0.     -0.5463]
 [-0.25    0.     -0.5463  0.      0.7963  0.    ]
 [ 0.     -0.5463  0.     -0.5463  0.      1.0926]]


(c) Calculate eigenvalues and eigenvectors of $\textbf{L}$ and present the data in one dimension. Remember to skip the smallest eigenvalue $\lambda \approx 0$. What is the corresponding clustering of cows?

---

The Fiedler vector provides a way to embed the data points (in this case, cows) into a one-dimensional space. The idea is that this one-dimensional representation captures the inherent structure of the data in a way that makes clustering more evident.

Here's how you can use the Fiedler vector for clustering:

One-dimensional Embedding: Each component of the Fiedler vector corresponds to a data point (cow). The value of each component provides a one-dimensional representation of that data point.

Thresholding: To cluster the data into two groups, you can use the sign of the components of the Fiedler vector. Data points with positive values in the Fiedler vector can be assigned to one cluster, while those with negative values can be assigned to another cluster. This is a simple binary clustering approach.

More than Two Clusters: If you want to cluster the data into more than two clusters, you can use additional eigenvectors (beyond the Fiedler vector) and perform k-means clustering on the multi-dimensional representation provided by these eigenvectors.

In essence, the Fiedler vector provides a way to represent the data in a lower-dimensional space (in this case, one-dimensional) where the clustering structure is more evident. By simply looking at the sign of the components of the Fiedler vector, you can achieve a binary clustering of the data.

In this exercise, the Fiedler vector is the eigenvector corresponding to the second smallest eigenvalue of the Laplacian matrix. The smallest eigenvalue is 0, and the corresponding eigenvector is a vector of ones. This eigenvector does not provide any useful information for clustering, and can be ignored. 

In [30]:
# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(L)

# Sort eigenvalues in ascending order and get the indices
sorted_indices = np.argsort(eigenvalues)

# Reorder eigenvalues and eigenvectors
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Fiedler vector (corresponding to the second smallest eigenvalue)
second_smallest_eigenvector = eigenvectors[:, 1]

# Cluster data based on the sign of the Fiedler vector components
clusters = np.sign(second_smallest_eigenvector)

print("The eigenvalues:")
print(eigenvalues)
print("\nThe eigenvectors:")
print(eigenvectors)

print("\nThe second smallest eigenvalue is:", eigenvalues[1])
print("The second smallest eigenvector is:")
print(second_smallest_eigenvector)

print("\nClustering (based on sign of the second smallest eigenvalue components):", clusters)
print("The clustering depends only on categorical features, so numerical features are ignored")

print("\nThe cow data looks like this")

print(
    """
Clover,Holstein,2,10,calm,rock
Sunny,Ayrshire,2,15,lively,country
Rose,Holstein,5,20,calm,classical
Daisy,Ayrshire,4,25,kind,rock
Strawberry,Finncattle,7,35,calm,classical
Molly,Ayrshire,8,45,kind,country
    """
)

The eigenvalues:
[-1.11022302e-16  1.76252371e-01  7.96300000e-01  1.09260000e+00
  1.63890000e+00  1.75894763e+00]

The eigenvectors:
[[ 0.40824829 -0.25110149 -0.42505624 -0.49910455 -0.28867513 -0.45305199]
 [ 0.40824829  0.49108747  0.53055701 -0.49910455 -0.28867513  0.17415595]
 [ 0.40824829 -0.44247163  0.19451594 -0.04230009  0.57735027  0.51421163]
 [ 0.40824829  0.25110149 -0.42505624  0.49910455 -0.28867513  0.45305199]
 [ 0.40824829 -0.49108747  0.53055701  0.49910455 -0.28867513 -0.17415595]
 [ 0.40824829  0.44247163  0.19451594  0.04230009  0.57735027 -0.51421163]]

The second smallest eigenvalue is: 0.17625237095193105
The second smallest eigenvector is:
[-0.25110149  0.49108747 -0.44247163  0.25110149 -0.49108747  0.44247163]

Clustering (based on sign of the second smallest eigenvalue components): [-1.  1. -1.  1. -1.  1.]
The clustering depends only on categorical features, so numerical features are ignored

The cow data looks like this

Clover,Holstein,2,10,calm,rock

(d) (Optional): Calculate the random-walk Laplacian $\mathbf{L}_{rw} = \mathbf{\Lambda}^{−1} \mathbf{L}$ and repeat step (c).

The Random Walk Laplacian is a variation of the graph Laplacian that is normalized differently. 

Here's the intuition behind the Random Walk Laplacian:

Random Walk on a Graph: Imagine a random walker moving around the graph. At each node, the walker randomly chooses an edge to traverse to the next node. The probability of choosing a particular edge is proportional to the weight of that edge relative to the total weight of all edges connected to the current node.

Transition Probability Matrix: The matrix $\mathbf{L}_{rw} = \mathbf{\Lambda}^{−1} \mathbf{L}$ can be thought of as a transition probability matrix for this random walk. The entry $\mathbf{L}rw_{ij}$ represents the probability of transitioning from node i to node j in one step.

Clustering Intuition: In the context of spectral clustering, the Random Walk Laplacian provides a way to cluster nodes that are more likely to be visited together during a random walk. If two nodes are in the same cluster, a random walker should frequently transition between them. Conversely, if two nodes are in different clusters, transitions between them should be infrequent.

In [33]:
# Calculate random-walk Laplacian matrix
Lrw = np.linalg.inv(Lambda) @ L

# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(Lrw)

# Sort eigenvalues in ascending order and get the indices
sorted_indices = np.argsort(eigenvalues)

# Reorder eigenvalues and eigenvectors
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Fiedler vector (corresponding to the second smallest eigenvalue)
second_smallest_eigenvector = eigenvectors[:, 1]

# Cluster data based on the sign of the Fiedler vector components
clusters = np.sign(second_smallest_eigenvector)

print("The eigenvalues:")
print(eigenvalues)
print("\nThe eigenvectors:")
print(eigenvectors)

print("\nThe second smallest eigenvalue is:", eigenvalues[1])
print("The second smallest eigenvector is:")
print(second_smallest_eigenvector)

print("\nClustering (based on sign of the second smallest eigenvalue components):", clusters)
print("The clustering depends only on categorical features, so numerical features are ignored")

print("\nThe cow data looks like this")

print(
    """
Clover,Holstein,2,10,calm,rock
Sunny,Ayrshire,2,15,lively,country
Rose,Holstein,5,20,calm,classical
Daisy,Ayrshire,4,25,kind,rock
Strawberry,Finncattle,7,35,calm,classical
Molly,Ayrshire,8,45,kind,country
    """
)

print("The clustering result is the same as the one obtained with the unnormalized Laplacian matrix")

The eigenvalues:
[-8.32667268e-17  1.89640609e-01  9.72126374e-01  1.32976665e+00
  1.67622397e+00  1.83224240e+00]

The eigenvectors:
[[-0.40824829  0.25253466  0.46566175 -0.49432852 -0.32199758  0.55829789]
 [-0.40824829 -0.48101314 -0.47862545 -0.5053325  -0.36840617 -0.13053406]
 [-0.40824829  0.45260646 -0.2325442  -0.01668449  0.51048453 -0.41384094]
 [-0.40824829 -0.25253466  0.46566175  0.49432852 -0.32199758 -0.55829789]
 [-0.40824829  0.48101314 -0.47862545  0.5053325  -0.36840617  0.13053406]
 [-0.40824829 -0.45260646 -0.2325442   0.01668449  0.51048453  0.41384094]]

The second smallest eigenvalue is: 0.18964060913962968
The second smallest eigenvector is:
[ 0.25253466 -0.48101314  0.45260646 -0.25253466  0.48101314 -0.45260646]

Clustering (based on sign of the second smallest eigenvalue components): [ 1. -1.  1. -1.  1. -1.]
The clustering depends only on categorical features, so numerical features are ignored

The cow data looks like this

Clover,Holstein,2,10,calm,rock